In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV

In [2]:
def metricas(model,Xt,Xv,yt,yv):
    print "ROC train:%.3f | ROC test:%.3f " %(roc_auc_score(y_true=yt,y_score=model.predict_proba(Xt)[:,1]),roc_auc_score(y_true=yv,y_score=model.predict_proba(Xv)[:,1]))
    print "ACC train:%.3f | ACC test:%.3f " %(accuracy_score(y_true=yt,y_pred=model.predict(Xt)),accuracy_score(y_true=yv,y_pred=model.predict(Xv)))

In [3]:
df = pickle.load(open('churn_con_woe','rb'))

In [4]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,...,W_V_ACCT_LGHT,W_V_N_VM_MESS,W_V_T_DAY_CHG,W_V_T_EVE_CALLS,W_V_T_EVE_CHG,W_V_T_NIG_CALLS,W_V_T_NIG_CHG,W_V_T_INT_CALLS,W_V_T_INT_CHG,W_V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,0.061033,1.219037,-1.864202,0.068283,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
1,156,No,No,0,29.67,116,16.78,86,10.73,2,...,0.061033,-0.147174,0.561314,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
2,57,No,No,0,14.60,127,16.48,93,10.42,2,...,0.053118,-0.147174,0.178109,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
3,58,No,Yes,20,33.07,89,18.16,92,10.65,2,...,0.053118,1.219037,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
4,149,No,No,0,31.89,81,17.12,79,11.89,1,...,0.061033,-0.147174,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956


In [5]:
X = df[[v for v in df.columns if v[:2]=='V_']+[v for v in df.columns if v[:4]=='W_C_']].copy()
y = df['TARGET'].copy()

In [6]:
pipe = make_pipeline(MinMaxScaler())
pipe.fit(X)
Xs = pd.DataFrame(pipe.transform(X),columns=X.columns)

In [7]:
Xs.head()

,V_ACCT_LGHT,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,V_T_INT_CHG,V_CS_CALLS,W_C_IP,W_C_VMP
0,0.633880,0.581395,0.841885,0.489362,0.492640,0.377778,0.693383,0.166667,0.500000,0.166667,1.0,1.0
1,0.786885,0.000000,0.484328,0.670213,0.492640,0.322222,0.667288,0.083333,0.544199,0.000000,1.0,0.0
2,0.245902,0.000000,0.134432,0.787234,0.477920,0.400000,0.638397,0.083333,0.508287,0.000000,1.0,0.0
3,0.251366,0.465116,0.563269,0.382979,0.560353,0.388889,0.659832,0.083333,0.464088,0.166667,1.0,1.0
4,0.748634,0.000000,0.535872,0.297872,0.509323,0.244444,0.775396,0.000000,0.411602,0.166667,1.0,0.0


In [8]:
Xt,Xv,yt,yv = train_test_split(Xs,y,train_size=0.7)

In [13]:
model = SGDClassifier(loss='log')

In [14]:
model.fit(Xt,yt)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [17]:
help(SGDClassifier)

Help on class SGDClassifier in module sklearn.linear_model.stochastic_gradient:

class SGDClassifier(BaseSGDClassifier, sklearn.feature_selection.from_model._LearntSelectorMixin)
 |  Linear classifiers (SVM, logistic regression, a.o.) with SGD training.
 |  
 |  This estimator implements regularized linear models with stochastic
 |  gradient descent (SGD) learning: the gradient of the loss is estimated
 |  each sample at a time and the model is updated along the way with a
 |  decreasing strength schedule (aka learning rate). SGD allows minibatch
 |  (online/out-of-core) learning, see the partial_fit method.
 |  For best results using the default learning rate schedule, the data should
 |  have zero mean and unit variance.
 |  
 |  This implementation works with data represented as dense or sparse arrays
 |  of floating point values for the features. The model it fits can be
 |  controlled with the loss parameter; by default, it fits a linear support
 |  vector machine (SVM).
 |  
 |  

In [15]:
metricas(model,Xt,Xv,yt,yv)

ROC train:0.809 | ROC test:0.772 
ACC train:0.851 | ACC test:0.822 


In [22]:
 param_grid = dict(loss = ['log'],
                  penalty = ['l1','l2','elasticnet'],
                  alpha = np.arange(0.0001,0.001,0.0001),
                  eta0 = np.arange(0.01,0.1,0.01),
                  learning_rate = ['constant','optimal','invscaling']
                  )

In [23]:
grid = GridSearchCV(cv=3,
                  verbose=True,
                  scoring='roc_auc',
                  estimator=model,
                  n_jobs=-1,
                  param_grid=param_grid)

In [24]:
grid.fit(Xs,y)

Fitting 3 folds for each of 729 candidates, totalling 2187 fits


[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1480 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 2187 out of 2187 | elapsed:   26.4s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2', 'elasticnet'], 'alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]), 'learning_rate': ['constant', 'optimal', 'invscaling'], 'eta0': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]), 'loss': ['log']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=True)

In [25]:
model = grid.best_estimator_
model.fit(Xt,yt)

metricas(model,Xt,Xv,yt,yv)

ROC train:0.801 | ROC test:0.775 
ACC train:0.876 | ACC test:0.859 
